**INSTALLING REQUIRED LIBRARIES**

In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 75.5 MB/s eta 0:00:00


**USE BNB_CONFIG TO REDUCE THE AMOUNT OF MEMORY USED**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

**LOAD MODEL FROM HUGGINGFACE**

In [ ]:
model_id = "google/gemma-2-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

**ADD TRAINING DATA**

In [ ]:
!unzip training_data.zip

Archive:  training_data.zip
   creating: 33/
  inflating: 33/data.json            
  inflating: 33/data.md              
   creating: 34/
  inflating: 34/data.json            
  inflating: 34/data.md              
   creating: 35/
  inflating: 35/data.json            
  inflating: 35/data.md              
   creating: 36/
  inflating: 36/data.json            
  inflating: 36/data.md              
   creating: 37/
  inflating: 37/data.json            
  inflating: 37/data.md              
   creating: 38/
  inflating: 38/data.json            
  inflating: 38/data.md              
   creating: 39/
  inflating: 39/data.json            
  inflating: 39/data.md              
   creating: 40/
  inflating: 40/data.json            
  inflating: 40/data.md              
   creating: 41/
  inflating: 41/data.json            
  inflating: 41/data.md              
   creating: 42/
  inflating: 42/data.json            
  inflating: 42/data.md              
   creating: 43/
  inflating: 43/data.json

In [ ]:
import os
import json
import pandas as pd

# Initialize the dictionary to store data
data = {
    "input": [],
    "output": []
}

# Iterate over each subdirectory and its files in the '/content' directory
for subdir, _, files in os.walk('/content'):
    json_content = None
    markdown_content = None

    # Look for .json and .md files in the current subdirectory
    for file in files:
        file_path = os.path.join(subdir, file)

        if file.endswith('.json') and os.path.isfile(file_path):
            with open(file_path, 'r') as f:
                json_content = json.load(f)
                # Check if json_content is a dictionary and extract the "annotation" key
                if isinstance(json_content, dict):
                    json_content = json_content.get("annotation")
                else:
                    json_content = None  # Handle case where JSON is not a dictionary

        elif file.endswith('.md') and os.path.isfile(file_path):
            with open(file_path, 'r') as f:
                markdown_content = f.read()

    # Append contents to the respective lists if both files are found
    if json_content and markdown_content:
        data["input"].append(json_content)
        data["output"].append(markdown_content)

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to verify structure
print(df)


                                                input  \
0   <data-piece><history><entry><input>sudo apt in...   
1   <data-piece><history><entry><input>sudo apt in...   
2   <data-piece><history><entry><input>find /home/...   
3   <data-piece><history><entry><input></input><ou...   
4   <data-piece><history><entry><input>grep 'patte...   
5   <data-piece><history><entry><input></input><ou...   
6   <data-piece><history><entry><input>rsync -av /...   
7   <data-piece><history><entry><input>git init my...   
8   <data-piece><history><entry><input>cp file1.tx...   
9   <data-piece><history><entry><input>sudo apt in...   
10  <data-piece><history><entry><input>sudo nano /...   
11  <data-piece><history><entry><input></input><ou...   
12  <data-piece><history><entry><input></input><ou...   
13  <data-piece><history><entry><input>tar -cvf ar...   
14  <data-piece><history><entry><input>top</input>...   
15  <history><entry><input>sudo apt install apache...   
16  <data-piece><history><entry

**GENERATE PROMPTS FOR EACH DATA POINT**

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")  # Replace with your model

# Function to generate the prompt
def generate_prompt(data_point):
    return f"""
        <start_of_turn>user
        Based on the following annotated Asciinema recording:
        {data_point["input"]}
        Generate a single cohesive Markdown file that explains and summarizes the entire process from the Asciinema recording.
        The summary should focus on:
        * Describing the context of each action.
        * Clearly explaining why the action was used, what the action does, and the outcome of this action.
        * Focus on making the documentation informative enough to be easily followed, capturing the step-by-step workflow as demonstrated in the recording, without using generic phrases.

        In the Markdown file, I want the following:
        * The main title should be preceded with a # and section titles should be preceded with ##.
        * End the markdown file with a 'Key Takeaways' section that summarizes the main points of the recording.
        <end_of_turn>\n
        <start_of_turn>model
        {data_point["output"]}<end_of_turn>
    """

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

dataset = dataset.train_test_split(test_size=0.2)
print(dataset)

train_data = dataset["train"]
test_data = dataset["test"]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 24
    })
    test: Dataset({
        features: ['input', 'output', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 6
    })
})


**APPLY LORA STUFF (MAKES FINE TUNING LESS EXPENSIVE)**

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

['gate_proj', 'q_proj', 'down_proj', 'k_proj', 'v_proj', 'up_proj', 'o_proj']


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 83066880 | total: 2697408768 | Percentage: 3.0795%


**TRAIN THE MODEL**

In [ ]:
import transformers

from trl import SFTTrainer


tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()


trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        #warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due 

In [ ]:
model.config.use_cache = False
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,1.616200
2,1.441900
3,0.902600
4,0.967900
5,0.662000
6,0.773900
7,0.594500
8,0.360600
9,0.415600
10,0.605200


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=100, training_loss=0.14379940164741128, metrics={'train_runtime': 3303.5229, 'train_samples_per_second': 0.121, 'train_steps_per_second': 0.03, 'total_flos': 4449271621690368.0, 'train_loss': 0.14379940164741128, 'epoch': 16.666666666666668})

**SAVE MODEL LOCALLY**

In [ ]:
new_model = "csc392-model-2-final"

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

**SAVE MODEL TO HUGGINGFACE**

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/cfstar188/csc392-model-2-final/commit/4771818a51f845bcb8de6b5e4677619800e29add', commit_message='Upload tokenizer', commit_description='', oid='4771818a51f845bcb8de6b5e4677619800e29add', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cfstar188/csc392-model-2-final', endpoint='https://huggingface.co', repo_type='model', repo_id='cfstar188/csc392-model-2-final'), pr_revision=None, pr_num=None)